In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

def trực quan hoá dữ liệu dự báo

In [ ]:
def Visualize_model_reg(y, y_pred):
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.scatter(y_pred, y)
    plt.xlabel('Model Prediction')
    plt.ylabel('True Vale')
    plt.plot([0, np.max(y) + 2*np.min(y)], [0, np.max(y) + 2*np.min(y)], '-', color = 'r')
    plt.subplot(1,2,2)
    sns.distplot(y, hist=False, color='r', label='True Value')
    sns.distplot(y_pred, hist=False, color='b', label='Model Prediction', axlabel='Distribution')
    plt.show()

In [ ]:
def Visualize_model_reg2(y_train, y_test, y_train_hat, y_test_hat):
    plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    ax1 = sns.distplot(y_train, hist=False, color="b", label='Train Actual')
    sns.distplot(y_train_hat, hist=False, color="r", label='Train Predict', ax=ax1)
    plt.subplot(1,2,2)
    ax2 = sns.distplot(y_test, hist=False, color="b", label='Test Actual')
    sns.distplot(y_test_hat, hist=False, color="r", label='Test Predict', ax=ax2)
    plt.show()

def tính toán so sánh kết quả dự báo và dữ liệu thật

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def Static_score_model_reg(y, y_pred):
    r2 = r2_score(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    return r2, mse, mae

Ghi model

In [ ]:
# lưu model và load model
# Ap dung duoc voi ca model dung trong feature engineering va model du doan cuoi cung
def Save_Object(obj, filename):
    import pickle
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)
    return
def Load_Object(filename):
    import pickle
    with open(filename, 'rb') as file:
        obj = pickle.load(file)
    return obj

In [ ]:
!pip install https://github.com/ydataai/pandas-profiling/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/ydataai/pandas-profiling/archive/master.zip (21.8 MB)


In [ ]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# statistics libraries
import numpy as np
import pandas as pd
# from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
# from mpl_toolkits.mplot3d import Axes3D
# from IPython.display import Image
from scipy.stats import itemfreq
from collections import Counter
from statsmodels.formula.api import ols
from scipy.stats import chi2_contingency, chi2
import statsmodels.api as sm
import scipy
from scipy import stats
import math
import time

# data pre-processing libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
# from imblearn.under_sampling import RandomUnderSampler

# Models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
# from pmdarima import auto_arima
from sklearn import svm
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
# import xgboost as xgb 

# tensorflow libraries to create ANN and build model
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers, Sequential, Input, Model
# from tensorflow.keras.layers import Dense
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.models import load_model
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.utils import plot_model
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# evaluation libraries
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, f1_score

# Turning parameter
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# saving model libraries
from sklearn.pipeline import Pipeline
import pickle

# funcsion libraries
import sys; 
import os
import re
# path=os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '\Lib'
# sys.path.insert(0,path)
# import import_ipynb
# from lib_evaluation_regression_model import *

import warnings
warnings.filterwarnings('ignore')

#Step 2: DATA UNDERSTANDING

In [ ]:
# create statistics function
def continous_feature_describe(lst_cols, df):
    col_ = []
    count_ = []
    min_ = []
    max_ = []
    mean_ = []
    std_ = []
    var_ = []
    skewness_ = []
    kurtosis_ = []
    do_lech = []
    do_nhon = []

    for i in lst_cols:
        col_.append(i)
        count_.append(df[i].count())
        min_.append(df[i].min())
        max_.append(df[i].max())
        mean_.append(df[i].mean())
        std_.append(df[i].std())
        var_.append(df[i].var())
        skewness_.append(df[i].skew())
        kurtosis_.append(df[i].kurtosis())
        
        if df[i].skew() > 0: do_lech.append('lech phai')
        elif df[i].skew() < 0: do_lech.append('lech trai')
        else: do_lech.append('doi xung')
            
        if df[i].kurtosis() > 0: do_nhon.append('nhon hon PP chuan')
        elif df[i].kurtosis() < 0: do_nhon.append('bet hon PP chuan')
        else: do_nhon.append('tuong duong PP chuan')

    df_describe = pd.DataFrame({'feature_name': col_,
                               'count': count_,
                               'min': min_,
                               'max': max_,
                               'mean': mean_,
                               'std': std_,
                               'var': var_,
                                'skewness': skewness_,
                                'kurtosis': kurtosis_,
                                'do_lech': do_lech,
                                'do_nhon': do_nhon
                               })
    return df_describe

#Step 4&5: MODELING & EVALUATION/ANALYZE

In [ ]:
# create Cross validation:
def Average_RMSE_Model(model, X, y, size=0.3, cv=10):
    import time
    from sklearn.model_selection import train_test_split    
    train_rmse=[]
    train_score=[]
    test_rmse=[]
    test_score=[]
    
    duration=[]
    for n in range(1, cv+1):                
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size)                                             
        
        start=time.time()        
        model.fit(X_train, y_train)
        end=time.time()
        
        train_rmse.append(mean_squared_error(y_true=y_train, y_pred=model.predict(X_train), squared=False))
        test_rmse.append(mean_squared_error(y_true=y_test, y_pred=model.predict(X_test), squared=False))
        train_score.append(model.score(X_train, y_train))
        test_score.append(model.score(X_test, y_test))

        duration.append((end-start)*1000)
        
    return np.mean(train_rmse), np.mean(train_score), np.mean(test_rmse), np.mean(test_score), np.mean(duration) 

In [ ]:
def Save_Object(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)
    return
def Load_Object(filename):
    with open(filename, 'rb') as file:
        obj = pickle.load(file)
    return obj

# Pipeline - Project 1

## Problem 1

### Create def

In [ ]:
def data_cleaning(data):
    df = data.copy()
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    # drop duplicates
    df = df.drop_duplicates() 
    # change datatype
    df['Date'] = pd.to_datetime(df['Date']) 
    # change column names
    df.columns = df.columns.str.replace(" ","_")
    df.columns = ['Date', 'AveragePrice', 'Total_Volume', 'PLU_4046','PLU_4225','PLU_4770',
          'Total_Bags', 'Small_Bags', 'Large_Bags', 'XLarge_Bags', 'type', 'year',
          'region']
    # remove some region: 
    df = df.loc[df['region'] != "TotalUS",:]
    return df

### Create fit_transform function

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class Pre_Processing(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('Khởi tạo đối tượng Pre-Processing')
    def fit(self, X, y=None):
        return self
    def transform(self, data, y=None):
        import pickle
        # load list categorical & continous variables
        lst_cate = pd.read_csv('model/lst_cate.csv').iloc[:,0].to_list()
        lst_cont = pd.read_csv('model/lst_cont.csv').iloc[:,0].to_list()

        # load encoder model
        encoder = pickle.load(open('model/OneHotEncoder.sav','rb'))

        # data cleaning
        df = data.copy()
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        
        # drop duplicates
        df = df.drop_duplicates() 

        ## change datatype
        # df['Date'] = pd.to_datetime(df['Date']) 

        ## change column names
        df.columns = df.columns.str.replace(" ","_")
        df.columns = ['Date', 'AveragePrice', 'Total_Volume', 'PLU_4046','PLU_4225','PLU_4770',
              'Total_Bags', 'Small_Bags', 'Large_Bags', 'XLarge_Bags', 'type', 'year',
              'region']

        # remove some region: 
        df = df.loc[df['region'] != "TotalUS",:]

        # apply encoder on categorial columns
        lst_encode = ['type','region']

        # arr = encoder.transform(df[lst_encode]).toarray()
        arr = encoder.transform(df[lst_encode])

        cols = []
        n = 0
        for i in encoder.categories_:
            for j in i[0:]: 
                t = 'oh_' + lst_encode[n] + '_' +str(j)
                t = t.replace('-', '_')
                cols.append(t)
            n = n+1
        df_oh_encode = pd.DataFrame(arr, columns=cols)
        
        # concat encoded columns to df
        lst_concat = lst_cont
        df = pd.concat([df_oh_encode.reset_index(), df.reset_index()], axis=1).iloc[:,1:]

       
        return df

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class Create_XY(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('Khởi tạo đối tượng Create_XY')
    def fit(self, X, y=None):
        return self
    def transform(self, data, y=None):
        import pickle
        # load list categorical & continous variables
        lst_cate = pd.read_csv('model/lst_cate.csv').iloc[:,0].to_list()
        lst_cont = pd.read_csv('model/lst_cont.csv').iloc[:,0].to_list()

        # create X, y
        lst_cont_fea = lst_cont
        # lst_cont_fea.remove('AveragePrice')
        df = data
        X = pd.concat([df.loc[:, df.columns.str.startswith("oh_")], \
               df[lst_cont_fea]], axis=1)
        
        y = df['AveragePrice']

        return X, y

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class Data_Standardizing(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('Khởi tạo đối tượng Data Standardizing')
    def fit(self, X, y=None):
        return self
    def transform(self, data, y=None):
        import pickle
        # load scaler model
        scaler = pickle.load(open('model/Problem1_RobustScaler.sav','rb'))

        # transform data
        lst_cont = pd.read_csv('model/lst_cont.csv').iloc[:,0].to_list()
        lst_cont_fea = lst_cont
        lst_cont_fea.remove('AveragePrice')   

        X = data.copy()
        X_before_scale = X[lst_cont_fea]
        X_scale = scaler.transform(X_before_scale) #--> dùng transform, không dùng fit_transform cho tập Test
        X_scale = pd.DataFrame(X_scale, columns=(X_before_scale.add_suffix('_scale')).columns)

        # concat data
        X_new = pd.concat([X.reset_index(drop=True), X_scale], \
                                axis=1)
        # select scaled features
        X_scale = pd.concat([X_new.loc[:, X_new.columns.str.startswith("oh_")], \
                                    X_new.loc[:, X_new.columns.str.endswith("_scale")]], \
                                    axis=1)
      
        return X_scale

## Problem 2,3

### Create def

In [ ]:
def RNN_LSTM_California_organic_forecast(data, n_head=1):
  # n_head = input('Input number of head:')
  # n_head = int(n_head)
  scaler = pickle.load(open('outputs/Problem2_scaler_RNN_LSTM_California_organic.sav', 'rb'))
  model = load_model('outputs/Problem2_RNN_LSTM_California_organic_model_one.h5')

  df = data.copy()
  df_California_organic = df.loc[(df['region']=='California')&(df['type']=='organic'),['Date','AveragePrice']].sort_values(by='Date')
  df_California_organic['Month'] = df_California_organic['Date'].to_numpy().astype('datetime64[M]')
  df_California_organic['Month'] = df_California_organic['Month'].astype('datetime64[ns]')
  df_California_organic_groupby = df_California_organic.groupby(['Month']).agg({'AveragePrice': np.mean})
  df_California_organic_groupby.index = pd.to_datetime(df_California_organic_groupby.index)

  df_California_organic_groupby.index.freq = 'MS'# frequent là month
  df_California_organic_groupby.index.name="DATE"

  dataframe = df_California_organic_groupby
  dataset_one = dataframe[['AveragePrice']]
  dataset_one = dataset_one.values.astype('float32')

  for i in range(n_head):
    x_new = dataset_one[-3:]
    x_new = scaler.transform(x_new) # the last value => predict next value
    x_new = x_new.reshape(1,1,3)

    y_new = model.predict(x_new)
    y_new = scaler.inverse_transform(y_new)
    dataset_one = np.vstack((dataset_one,y_new ))
  return dataset_one

In [ ]:
def RNN_LSTM_California_conventional_forecast(data, n_head=1):
  # n_head = input('Input number of head:')
  # n_head = int(n_head)
  scaler = pickle.load(open('outputs/Problem3_scaler_RNN_LSTM_California_conventional.sav', 'rb'))
  model = load_model('outputs/Problem3_RNN_LSTM_California_conventional_model_one.h5')

  df = data.copy()
  df_California_conventional = df.loc[(df['region']=='California')&(df['type']=='conventional'),['Date','AveragePrice']].sort_values(by='Date')
  df_California_conventional['Month'] = df_California_conventional['Date'].to_numpy().astype('datetime64[M]')
  df_California_conventional['Month'] = df_California_conventional['Month'].astype('datetime64[ns]')
  df_California_conventional_groupby = df_California_conventional.groupby(['Month']).agg({'AveragePrice': np.mean})
  df_California_conventional_groupby.index = pd.to_datetime(df_California_conventional_groupby.index)

  df_California_conventional_groupby.index.freq = 'MS'# frequent là month
  df_California_conventional_groupby.index.name="DATE"

  dataframe = df_California_conventional_groupby
  dataset_one = dataframe[['AveragePrice']]
  dataset_one = dataset_one.values.astype('float32')

  for i in range(n_head):
    x_new = dataset_one[-3:]
    x_new = scaler.transform(x_new) # the last value => predict next value
    x_new = x_new.reshape(1,1,3)

    y_new = model.predict(x_new)
    y_new = scaler.inverse_transform(y_new)
    dataset_one = np.vstack((dataset_one,y_new ))
  return dataset_one

In [ ]:
def FP_California_organic_forecast(df, n_head):
  df_California_organic = df.loc[(df['region']=='California')&(df['type']=='organic'),['Date','AveragePrice']].sort_values(by='Date')
  df_California_organic['Month'] = df_California_organic['Date'].to_numpy().astype('datetime64[M]')
  df_California_organic['Month'] = df_California_organic['Month'].astype('datetime64[ns]')
  df_California_organic_groupby = df_California_organic.groupby(['Month']).agg({'AveragePrice': np.mean})
  df_California_organic_groupby.index = pd.to_datetime(df_California_organic_groupby.index)

  df_California_organic_groupby.index.freq = 'MS'# frequent là month
  df_California_organic_groupby.index.name="DATE"
  df = df_California_organic_groupby.reset_index()  
  df.columns = ['ds','y']

  m = Prophet(yearly_seasonality=True, \
              daily_seasonality=False, weekly_seasonality=False) 
  m.fit(df)
  future = m.make_future_dataframe(periods=n_head, freq='M') # next 5 years
  forecast = m.predict(future)

  return m, forecast

In [ ]:
def FP_California_conventional_forecast(df, n_head):
  df_California_conventional = df.loc[(df['region']=='California')&(df['type']=='conventional'),['Date','AveragePrice']].sort_values(by='Date')
  df_California_conventional['Month'] = df_California_conventional['Date'].to_numpy().astype('datetime64[M]')
  df_California_conventional['Month'] = df_California_conventional['Month'].astype('datetime64[ns]')
  df_California_conventional_groupby = df_California_conventional.groupby(['Month']).agg({'AveragePrice': np.mean})
  df_California_conventional_groupby.index = pd.to_datetime(df_California_conventional_groupby.index)

  df_California_conventional_groupby.index.freq = 'MS'# frequent là month
  df_California_conventional_groupby.index.name="DATE"
  df = df_California_conventional_groupby.reset_index()  
  df.columns = ['ds','y']

  m = Prophet(yearly_seasonality=True, \
              daily_seasonality=False, weekly_seasonality=False) 
  m.fit(df)
  future = m.make_future_dataframe(periods=n_head, freq='M') # next 5 years
  forecast = m.predict(future)

  return m, forecast